In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras import Input
import keras.activations as act
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error 

In [2]:
df = pd.read_csv('./melting-point/train_wz_ws.csv')
df

,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,Group 8,Group 9,...,Group 406,Group 407,Group 408,Group 409,Group 410,Group 412,Group 414,Group 415,Group 416,Group 418
0,213.15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,407.15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,324.15,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,351.15,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,126.15,2,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2560,256.45,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2561,231.15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2562,256.25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2563,333.15,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_test = pd.read_csv('./melting-point/test.csv')
df_test = df_test[df_test.columns.intersection(df.columns)]
df_test

,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,Group 8,Group 9,Group 10,...,Group 406,Group 407,Group 408,Group 409,Group 410,Group 412,Group 414,Group 415,Group 416,Group 418
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,7,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,2,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
662,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
663,2,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
664,1,1,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_submission = pd.read_csv('./melting-point/sample_submission.csv')
df_submission

,id,Tm
0,1022,0
1,1146,0
2,79,0
3,2279,0
4,1342,0
...,...,...
661,2663,0
662,624,0
663,2655,0
664,2089,0


In [8]:
ss = StandardScaler()

melting_point={}
melting_point["data"] = ss.fit_transform(df.drop(['Tm'], axis=1).values)
melting_point["target"] = df["Tm"].values
melting_point["test"] = ss.fit_transform(df_test.values)

In [6]:
rows = len(melting_point['data'][0])

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(melting_point["data"], melting_point["target"], test_size=0.1)

In [9]:
def build_model(lr=0.001):
    model = Sequential()

    #337
    model.add(Input(shape=(rows,)))

    model.add(Dense(rows))
    model.add(Activation(act.sigmoid))

    model.add(Dense(rows//2))
    model.add(Activation(act.leaky_relu))

    model.add(Dense(1))
    
    model.compile(optimizer=Adam(learning_rate=lr), loss='mean_absolute_error', metrics=['r2_score'])
    return model

regressor = KerasRegressor(model=build_model, epochs=100, batch_size=32, verbose=1, lr=0.001)

In [16]:
regressor.fit(melting_point["data"], 
          melting_point["target"], 
          epochs=100, 
          batch_size=64,
          validation_data=(melting_point["data"], melting_point["target"]))

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 232.6365 - r2_score: -10.7788 - val_loss: 173.2044 - val_r2_score: -5.9243
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 101.0291 - r2_score: -2.0912 - val_loss: 54.2745 - val_r2_score: 0.0826
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 47.9970 - r2_score: 0.2615 - val_loss: 41.1526 - val_r2_score: 0.4353
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 39.2224 - r2_score: 0.4873 - val_loss: 35.5319 - val_r2_score: 0.5645
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 34.8343 - r2_score: 0.5794 - val_loss: 32.1766 - val_r2_score: 0.6303
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 32.1100 - r2_score: 0.6345 - val_loss: 29.9242 - val_r2_score: 0.6689
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.0425 - r2_score: 0.6696 - val_loss: 28.6156 - val_r2_score: 0.6956
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 28.9250 - r2_score: 0.6879 - va

,model,<function bui...001BDA0DF1E40>
,build_fn,None
,warm_start,False
,random_state,None
,optimizer,'rmsprop'
,loss,None
,metrics,None
,batch_size,32
,validation_batch_size,None
,verbose,1
,callbacks,None


In [17]:
pred = regressor.predict(melting_point["test"])

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [18]:
df_submission['Tm'] = pred
df_submission

,id,Tm
0,1022,341.725525
1,1146,341.804565
2,79,223.793823
3,2279,206.622208
4,1342,271.018585
...,...,...
661,2663,266.191711
662,624,265.637939
663,2655,177.698441
664,2089,240.984390


In [19]:
df_submission.to_csv('./melting-point/submission2.csv', index=False)